In [1]:
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
%run ../preprocessing.py

In [3]:
# Expand cycles so that there is a line per date (active or not) with a boolean indicator of period
cycles_processed = expand_cycles(cycles)

In [4]:
# Expand tracking so that there is a line per date (active or not) with a one hot encoded symtoms
tracking_processed = process_tracking(tracking)

In [5]:
# Merge cycles and tracking information
training = pd.merge(cycles_processed, tracking_processed, left_index=True, right_index=True, how='outer').fillna(0)

In [6]:
# Find the first day the user started using the app
training = pd.merge(
    training,
    cycles.groupby('user_id')\
          .agg({'cycle_start': {'first_use': 'min'}})\
          .reset_index()\
          .set_index('user_id')['cycle_start'],
    left_index=True,
    right_index=True
)

In [7]:
# Find the absolute day for each row from the day the user started using the app
absolute_day = (training.reset_index().date.dt.date - training.reset_index().first_use.dt.date).dt.days + 1
absolute_day.index = training.index
training['absolute_day'] = absolute_day

In [8]:
# Keep only the columns needed by the RNN
training = training[training_columns]

In [9]:
# Do a train/test split of the data
train_users = users.user_id.sample(frac=0.8)
training = training.reset_index()
df_train = training[training.user_id.isin(train_users)][training_columns]
df_test = training[~training.user_id.isin(train_users)][training_columns]